In [2]:
import pickle
import json

with open("/home/wsr217/mlama/output/results/mbert_base/en/P17/result.pkl", 'rb') as f:
    mlama_results = pickle.load(f)

with open("/home/wsr217/mpararel/data/mpararel_predictions/mbert_cased_with_mlama_2/en/P17.jsonl") as f:
    mpararel_results = json.load(f)

In [48]:
import sys
sys.path.append('../')
from dataset.cleanup import VALID_RELATIONS, clean_template
from evaluate_consistency.get_model_predictions import build_model_by_name, get_populated_template, get_candidates_probabilities
from collections import defaultdict

In [4]:
def compare_ranks_mpararel_mlama(mlama_results, mpararel_results):
    same_rank = 0
    tuples_not_in_mpararel = 0
    tuples_in_mpararel = 0
    mpararel_correct = 0
    mlama_correct = 0
    tuples_where_different = []
    for mlama_result in mlama_results['list_of_results']:
        sample = mlama_result['sample']
        tuple_key = "{}-{}".format(sample["sub_label"], sample["obj_label"])
        if tuple_key in mpararel_results:
            mlama_template = sample['masked_sentences'][0]
            mlama_template = mlama_template.replace("[MASK]", "[Y]")
            mlama_template = mlama_template.replace(sample["sub_label"], "[X]")
            mlama_template = clean_template(mlama_template)
            if tuple_key not in mpararel_results:
                tuples_not_in_mpararel += 1
                continue
            tuples_in_mpararel += 1
            mpararel_result = [(template, pred, rank) for template, pred, rank in
                            mpararel_results[tuple_key] if template == mlama_template]
            if len(mpararel_result) == 0:
                print("Error: mLAMA template '{}' not found in tuple '{}'".format(
                    mlama_template, tuple_key))
                continue
            mpararel_rank = int(mpararel_result[0][2])
            mpararel_correct += 1 if mpararel_rank == 0 else 0
            mlama_correct += 1 if mlama_result['masked_topk']['rank'] == 0 else 0
            if mlama_result['masked_topk']['rank'] == mpararel_rank:
                same_rank += 1
            else:
                tuples_where_different.append((sample["sub_label"], sample["obj_label"]))
    return same_rank, tuples_in_mpararel, tuples_not_in_mpararel, mpararel_correct, mlama_correct, tuples_where_different

In [7]:
relation_to_tuples_where_different = {}
for relation in VALID_RELATIONS:
    with open(f"/home/wsr217/mlama/output/results/mbert_base/en/{relation}/result.pkl", 'rb') as f:
        mlama_results = pickle.load(f)

    with open(f"/home/wsr217/mpararel/data/mpararel_predictions/mbert_cased_with_mlama_2/en/{relation}.jsonl") as f:
        mpararel_results = json.load(f)
    
    print("relation:", relation)
    (same_rank,
     tuples_in_mpararel,
     tuples_not_in_mpararel,
     mpararel_correct,
     mlama_correct, tuples_where_different) = compare_ranks_mpararel_mlama(mlama_results, mpararel_results)
    relation_to_tuples_where_different[relation] = tuples_where_different
    print("same_rank={} tuples_in_mpararel={} "
          "tuples_not_in_mpararel={} mpararel_correct={} mlama_correct={}".format(
              same_rank, tuples_in_mpararel, tuples_not_in_mpararel,
              mpararel_correct, mlama_correct))
    print("Same {0:0.1%}".format(same_rank/tuples_in_mpararel),
          mpararel_correct>mlama_correct,
          "{0:0.1%}".format(mpararel_correct/tuples_in_mpararel))
    print()

relation: P276
same_rank=311 tuples_in_mpararel=936 tuples_not_in_mpararel=0 mpararel_correct=704 mlama_correct=388
Same 33.2% True 75.2%

relation: P140
same_rank=175 tuples_in_mpararel=451 tuples_not_in_mpararel=0 mpararel_correct=9 mlama_correct=7
Same 38.8% True 2.0%

relation: P136
same_rank=504 tuples_in_mpararel=923 tuples_not_in_mpararel=0 mpararel_correct=805 mlama_correct=503
Same 54.6% True 87.2%

relation: P131
same_rank=156 tuples_in_mpararel=864 tuples_not_in_mpararel=0 mpararel_correct=617 mlama_correct=183
Same 18.1% True 71.4%

relation: P449
same_rank=404 tuples_in_mpararel=830 tuples_not_in_mpararel=0 mpararel_correct=368 mlama_correct=306
Same 48.7% True 44.3%

relation: P178
same_rank=354 tuples_in_mpararel=571 tuples_not_in_mpararel=0 mpararel_correct=562 mlama_correct=352
Same 62.0% True 98.4%

relation: P39
same_rank=164 tuples_in_mpararel=863 tuples_not_in_mpararel=0 mpararel_correct=461 mlama_correct=160
Same 19.0% True 53.4%

relation: P279
same_rank=471 tupl

In [9]:
relation = "P178"
mlama_template = "[X] is developed by [Y] ."
mpararel_template = "[X] is developed by [Y]"

In [10]:
tuples_where_different = relation_to_tuples_where_different[relation]
tuples_where_different

[('MessagePad', 'Apple Inc.'),
 ('macOS', 'Apple Inc.'),
 ('iOS', 'Apple Inc.'),
 ('Safari', 'Apple Inc.'),
 ('iLife', 'Apple Inc.'),
 ('iWork', 'Apple Inc.'),
 ('PostScript', 'Adobe'),
 ('MS-DOS', 'Microsoft'),
 ('GW-BASIC', 'Microsoft'),
 ('ASIMO', 'Honda'),
 ('Xenix', 'Microsoft'),
 ('MSX-DOS', 'Microsoft'),
 ('MiniDisc', 'Sony'),
 ('ISPF', 'IBM'),
 ('MobileMe', 'Apple Inc.'),
 ('Zaxxon', 'Sega'),
 ('NetBurst', 'Intel'),
 ('Symbian', 'Nokia'),
 ('MacPaint', 'Apple Inc.'),
 ('QuickTime', 'Apple Inc.'),
 ('XScale', 'Intel'),
 ('ThinkPad', 'IBM'),
 ('iPod', 'Apple Inc.'),
 ('HyperTalk', 'Apple Inc.'),
 ('HyperCard', 'Apple Inc.'),
 ('Gmail', 'Google'),
 ('Xcode', 'Apple Inc.'),
 ('REXX', 'IBM'),
 ('Newsstand', 'Apple Inc.'),
 ('Zune', 'Microsoft'),
 ('iTunes', 'Apple Inc.'),
 ('FairPlay', 'Apple Inc.'),
 ('iChat', 'Apple Inc.'),
 ('SonicStage', 'Sony'),
 ('TSS/360', 'IBM'),
 ('Yakuza', 'Sega'),
 ('iGoogle', 'Google'),
 ('Picasa', 'Google'),
 ('VisualAge', 'IBM'),
 ('QuickDraw', 'Apple 

In [11]:
tuple_index = 6
with open(f"/home/wsr217/mlama/output/results/mbert_base/en/{relation}/result.pkl", 'rb') as f:
    mlama_results = pickle.load(f)
    for mlama_result in mlama_results['list_of_results']:
        sample = mlama_result['sample']
        if (sample["sub_label"] == tuples_where_different[tuple_index][0] and
                sample["obj_label"] == tuples_where_different[tuple_index][1]):
            print(mlama_result)

with open(f"/home/wsr217/mpararel/data/mpararel_predictions/mbert_cased_with_mlama_2/en/{relation}.jsonl") as f:
    mpararel_results = json.load(f)
    tuple_key = "{}-{}".format(tuples_where_different[tuple_index][0],
                               tuples_where_different[tuple_index][1])
    print([(t,p,r) for t,p,r in mpararel_results[tuple_key] if t == mpararel_template])

{'sample': {'sub_label': 'PostScript', 'obj_label': 'Adobe', 'uuid': 6, 'masked_sentences': ['PostScript is developed by [MASK] .', 'PostScript is developed by [MASK] [MASK] .', 'PostScript is developed by [MASK] [MASK] [MASK] .']}, 'uuid': 6, 'token_ids': array([  101,   177, 11369, 46072, 10124, 14628, 10155,   103,   119,
         102,     0,     0,     0]), 'masked_indices': array([list([7]), list([7, 8]), list([7, 8, 9])], dtype=object), 'label_index': [17216, 13604, 119], 'masked_topk': {'rank': 2, 'prob_true': -5.395622, 'predicted': array(['Airbus basic', 'Access', 'Square', 'NATO', 'Honda', 'Fairchild',
       'Mitsubishi', 'Gibson', 'British Broadcasting Corporation',
       'Douglas', 'Yahoo', 'Apple Inc.', 'Sega', 'Nintendo', 'Google',
       'Atari', 'Nokia', 'Boeing', 'Sony', 'Amazon', 'Intel', 'Samsung',
       'Adobe', 'IBM', 'Microsoft'], dtype='<U32'), 'probs': array([-11.683069 , -11.326002 , -10.417402 ,  -9.07726  ,  -8.563734 ,
        -8.428636 ,  -8.121189 ,  -8

In [12]:
model, tokenizer = build_model_by_name('bert-base-multilingual-cased', 'cpu')

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [27]:
mlama_candidates = set([r['sample']['obj_label'] for r in mlama_results['list_of_results']])
len(mlama_candidates)

25

In [28]:
max_tokens_count = 0
tokens_count_to_obj_to_ids = defaultdict(lambda: defaultdict(list))
for candidate_obj in mlama_candidates:
    tokens = tokenizer.tokenize(candidate_obj)
    ids = tokenizer.convert_tokens_to_ids(tokens)
    tokens_count_to_obj_to_ids[len(
        tokens)][candidate_obj] = ids
    max_tokens_count = max(max_tokens_count, len(tokens))
max_tokens_count

3

In [29]:
tuples_where_different[tuple_index]

('PostScript', 'Adobe')

In [31]:
import torch

In [51]:
mlama_inputs = []
mpararel_inputs = []
mpararel_with_space_inputs = []
mpararel_with_point_inputs = []
mpararel_with_space_and_point_inputs = []
for masks_count in tokens_count_to_obj_to_ids.keys():
    mlama_inputs.append(
        get_populated_template(
            mlama_template, {"obj_label":tuples_where_different[tuple_index][0]},
            tokenizer.mask_token, masks_count))
    mpararel_inputs.append(
        get_populated_template(
            mpararel_template, {"obj_label":tuples_where_different[tuple_index][0]},
            tokenizer.mask_token, masks_count))
    mpararel_with_space_inputs.append(
        get_populated_template(
            mpararel_template + " ", {"obj_label":tuples_where_different[tuple_index][0]},
            tokenizer.mask_token, masks_count))
    mpararel_with_point_inputs.append(
        get_populated_template(
            mpararel_template + ".", {"obj_label":tuples_where_different[tuple_index][0]},
            tokenizer.mask_token, masks_count))
    mpararel_with_space_and_point_inputs.append(
        get_populated_template(
            mpararel_template + " .", {"obj_label":tuples_where_different[tuple_index][0]},
            tokenizer.mask_token, masks_count))
all_inputs = [mlama_inputs, mpararel_inputs, mpararel_with_space_inputs,
              mpararel_with_point_inputs, mpararel_with_space_and_point_inputs]
all_inputs

[['PostScript is developed by [MASK] .',
  'PostScript is developed by [MASK] [MASK] [MASK] .',
  'PostScript is developed by [MASK] [MASK] .'],
 ['PostScript is developed by [MASK]',
  'PostScript is developed by [MASK] [MASK] [MASK]',
  'PostScript is developed by [MASK] [MASK]'],
 ['PostScript is developed by [MASK] ',
  'PostScript is developed by [MASK] [MASK] [MASK] ',
  'PostScript is developed by [MASK] [MASK] '],
 ['PostScript is developed by [MASK].',
  'PostScript is developed by [MASK] [MASK] [MASK].',
  'PostScript is developed by [MASK] [MASK].'],
 ['PostScript is developed by [MASK] .',
  'PostScript is developed by [MASK] [MASK] [MASK] .',
  'PostScript is developed by [MASK] [MASK] .']]

In [56]:
for inputs in all_inputs:
    print(">>>")
    print(inputs)
    encoded_input = tokenizer(inputs, padding=True, return_tensors='pt')
    mask_indexes = torch.where(encoded_input.input_ids == tokenizer.mask_token_id, 1, 0)
    with torch.no_grad():
        output = model(**encoded_input)

    candidate_to_prob = {}
    masks_count = list(tokens_count_to_obj_to_ids.keys())
    for i in range(len(mask_indexes)):
        candidates_with_x_masks_to_prob = get_candidates_probabilities(
            output.logits[i], mask_indexes[i], tokens_count_to_obj_to_ids[masks_count[i]])
        for candidate, prob in candidates_with_x_masks_to_prob.items():
            candidate_to_prob[candidate] = prob
    predictions = sorted(candidate_to_prob.items(), key=lambda x: x[1], reverse=True)
    print('\n'.join([str(e)+", "+str(p) for e,p in predictions[:10]]))
    print()

>>>
['PostScript is developed by [MASK] .', 'PostScript is developed by [MASK] [MASK] [MASK] .', 'PostScript is developed by [MASK] [MASK] .']
Microsoft, 7.675020217895508
IBM, 7.227211952209473
Adobe, 6.75423526763916
Samsung, 6.543992042541504
Intel, 6.400230884552002
Amazon, 6.3362836837768555
Sony, 6.265348434448242
Boeing, 6.118687152862549
Nokia, 5.963374137878418
Atari, 5.760123252868652

>>>
['PostScript is developed by [MASK]', 'PostScript is developed by [MASK] [MASK] [MASK]', 'PostScript is developed by [MASK] [MASK]']
Apple Inc., 7.5920186042785645
Microsoft, 7.129244804382324
IBM, 6.814316749572754
Adobe, 6.507143020629883
Samsung, 6.421038627624512
Intel, 6.307311534881592
Nokia, 5.841700553894043
Google, 5.831084728240967
Amazon, 5.561495304107666
Sony, 5.550632476806641

>>>
['PostScript is developed by [MASK] ', 'PostScript is developed by [MASK] [MASK] [MASK] ', 'PostScript is developed by [MASK] [MASK] ']
Apple Inc., 7.5920186042785645
Microsoft, 7.129244804382324
IB

### mpararel

In [50]:
encoded_input = tokenizer(mpararel_inputs, padding=True, return_tensors='pt')
mask_indexes = torch.where(encoded_input.input_ids == tokenizer.mask_token_id, 1, 0)
mask_indexes

tensor([[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0]])

In [109]:
with torch.no_grad():
    output = model(**encoded_input)

In [110]:
output.logits.shape

torch.Size([3, 11, 119547])

In [111]:
candidate_to_prob = {}
for i in range(len(mask_indexes)):
    candidates_with_x_masks_to_prob = get_candidates_probabilities(
        output.logits[i], mask_indexes[i], tokens_count_to_obj_to_ids[i+1])
    for candidate, prob in candidates_with_x_masks_to_prob.items():
        candidate_to_prob[candidate] = prob
sorted(candidate_to_prob.items(), key=lambda x: x[1], reverse=True)

[('Apple Inc.', 7.5920186042785645),
 ('Microsoft', 7.129244804382324),
 ('IBM', 6.814316749572754),
 ('Adobe', 6.507143020629883),
 ('Samsung', 6.421038627624512),
 ('Intel', 6.307311534881592),
 ('Nokia', 5.841700553894043),
 ('Google', 5.831084728240967),
 ('Amazon', 5.561495304107666),
 ('Sony', 5.550632476806641),
 ('Nintendo', 5.407355308532715),
 ('Boeing', 5.32455587387085),
 ('Sega', 5.307690143585205),
 ('Atari', 5.016188621520996),
 ('Yahoo', 4.13648796081543),
 ('Douglas', 3.948868751525879),
 ('British Broadcasting Corporation', 3.9222687085469565),
 ('Honda', 3.671762704849243),
 ('Mitsubishi', 3.663083553314209),
 ('Fairchild', 3.3359124660491943),
 ('NATO', 3.251843214035034),
 ('Gibson', 1.9900282621383667),
 ('Airbus basic', 1.2176499366760254),
 ('Square', 0.9993970394134521),
 ('Access', 0.9894540905952454)]

## Compare mpararel and pararel templates in en

In [64]:
import os
import collections

relation_to_templates = collections.defaultdict(set)
mpararel_folder = "/home/wsr217/mpararel/data/mpararel_with_mlama/patterns/en"
for relation_file in os.listdir(mpararel_folder):
    with open(os.path.join(mpararel_folder, relation_file)) as f:
        for line in f:
            line_data = json.loads(line)
            template = line_data["pattern"]
            template += "."
            relation_to_templates[relation_file].add(template)

pararel_folder = "/home/wsr217/mpararel/data/pararel/pattern_data/graphs_json"
for relation_file in os.listdir(pararel_folder):
    with open(os.path.join(pararel_folder, relation_file)) as f:
        for line in f:
            line_data = json.loads(line)
            #template = clean_template(line_data["pattern"])
            template = line_data["pattern"]
            if template in relation_to_templates[relation_file]:
                relation_to_templates[relation_file].remove(template)
            else:
                print(relation_file, "doesn't contain the template:", template)

P279.jsonl doesn't contain the template: [X],  a subclass of [Y].
P279.jsonl doesn't contain the template: [X],  a type of [Y].
P39.jsonl doesn't contain the template: [X] has the position of [Y] .
P1303.jsonl doesn't contain the template: [X] plays [Y] .
P1303.jsonl doesn't contain the template: [Y] player [X] 
P1303.jsonl doesn't contain the template: [X] plays the [Y] .
P101.jsonl doesn't contain the template: The expertise of [X] is [Y].
P101.jsonl doesn't contain the template: The domain of activity of [X] is [Y].
P101.jsonl doesn't contain the template: The domain of work of [X] is [Y].
P1412.jsonl doesn't contain the template: [X] used to communicate in [Y] .
P1412.jsonl doesn't contain the template: [X] communicated in [Y] .
P1412.jsonl doesn't contain the template: [X] typically used [Y] to communicate .
P1412.jsonl doesn't contain the template: [X] used [Y] to communicate .
P1412.jsonl doesn't contain the template: [Y] was a language used by [X] .
P1412.jsonl doesn't contain 

In [62]:
for relation, left_templates in relation_to_templates.items():
    if len(left_templates) > 0:
        print(relation, "has some templates left")